In [1]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [2]:
! $k get all

NAME                              READY   STATUS    RESTARTS   AGE
pod/hello-node-64c578bdf8-swwlq   1/1     Running   0          6m5s

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
service/hello-node   NodePort    10.152.183.71   <none>        7777:30080/TCP   4m20s
service/kubernetes   ClusterIP   10.152.183.1    <none>        443/TCP          21m

NAME                         READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello-node   1/1     1            1           6m5s

NAME                                    DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-node-64c578bdf8   1         1         1       6m5s


In [3]:
! microk8s.enable dns

Enabling DNS
Applying manifest
service/kube-dns unchanged
serviceaccount/kube-dns unchanged
configmap/kube-dns unchanged
deployment.extensions/kube-dns configured
Restarting kubelet
DNS is enabled


In [4]:
! microk8s.status 

microk8s is running
addons:
jaeger: disabled
fluentd: disabled
gpu: disabled
storage: disabled
registry: disabled
ingress: disabled
dns: enabled
metrics-server: disabled
prometheus: disabled
istio: disabled
dashboard: disabled


In [5]:
# ! microk8s.enable metallb

In [6]:
! $k create deployment hello-node --image=gcr.io/hello-minikube-zero-install/hello-node
! $k expose deployment hello-node --type=LoadBalancer --port=7777

Error from server (AlreadyExists): deployments.apps "hello-node" already exists
Error from server (AlreadyExists): services "hello-node" already exists


In [7]:
! $k get services

NAME         TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
hello-node   NodePort    10.152.183.71   <none>        7777:30080/TCP   4m28s
kubernetes   ClusterIP   10.152.183.1    <none>        443/TCP          21m


In [8]:
! sudo $k port-forward svc/hello-node 80:80 --address 10.12.69.17

error: Service hello-node does not have a service port 80


In [9]:
! $k get services

NAME         TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
hello-node   NodePort    10.152.183.71   <none>        7777:30080/TCP   4m28s
kubernetes   ClusterIP   10.152.183.1    <none>        443/TCP          21m


# Reference

https://github.com/tonybutzer/jup/blob/2dde57ebb5cf291e4def7aa120c31226f821e16c/juphub/labs/00-just-product/02-two-loadbalanced/Makefile

```
# Proxy
# http://kubernetes_master_address/api/v1/namespaces/namespace_name/services/[https:]service_name[:port_name]/proxy
# 

help:
	echo http://localhost:8001/api/v1/namespaces/default/services/product-service:8081/proxy/

# ubuntu@ip-10-12-69-193:~$ sudo kubectl port-forward svc/product-service 8080:8081 --address 10.12.69.193

```

In [10]:
! $k delete service hello-node

service "hello-node" deleted


In [11]:
! $k expose deployment hello-node --type=NodePort --port=7777

service/hello-node exposed


In [12]:
# ! sudo $k port-forward svc/hello-node 80:7777 --address 10.12.69.195

Forwarding from 10.12.69.195:80 -> 7777
^C


In [14]:
! $k get services

NAME         TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE
hello-node   NodePort    10.152.183.6   <none>        7777:30817/TCP   117s
kubernetes   ClusterIP   10.152.183.1   <none>        443/TCP          23m


In [15]:
! netstat -anlp |grep 7777

(Not all processes could be identified, non-owned process info
 will not be shown, you would have to be root to see it all.)
unix  2      [ ]         DGRAM                    17777    -                    


In [17]:
! curl 10.152.183.6:30817

curl: (7) Failed to connect to 10.152.183.6 port 30817: Connection timed out
